In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

In [2]:
# 载入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 参数概要
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("historm", var)

# 初始化权重
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正态分布
    var = tf.Variable(initial)
    variable_summaries(var)
    return var
# 初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    var = tf.Variable(initial)
    variable_summaries(var)
    return var

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784])
    y = tf.placeholder(tf.float32, [None, 10])

    # 改变x的格式，转为4D向量 [batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x, [-1, 28, 28, 1])

tf.summary.image('input', x_image, 10)

with tf.name_scope("conv1"):
    # 第一个卷积层
    with tf.name_scope("W_conv1"):
        W_conv1 = weight_variable([5, 5, 1, 32])
    with tf.name_scope("b_conv1"):
        b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope("conv2"):
    # 第二个卷积层
    with tf.name_scope("W_conv2"):
        W_conv2 = weight_variable([5, 5, 32, 64])
    with tf.name_scope("b_conv2"):
        b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

with tf.name_scope("fc1"):
    # 第一个全连接层
    with tf.name_scope("W_fc1"):
        W_fc1 = weight_variable([7*7*64, 1024])
    with tf.name_scope("b_fc1"):
        b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

with tf.name_scope("dropout"):
    # dropout层
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
with tf.name_scope("fc2"):
    # 第二个全连接层
    with tf.name_scope("W_fc2"):
        W_fc2 = weight_variable([1024, 10])
    with tf.name_scope("b_fc2"):
        b_fc2 = bias_variable([10])
    # h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    
    # 计算输出
    prediction = tf.matmul(h_fc1_drop, W_fc2) + b_fc2



with tf.name_scope("loss"):

    # 交叉熵代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    tf.summary.scalar("loss", loss)

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

with tf.name_scope("accuracy"):
    # 结果存放在一个Bool型列表中
    correct_predition = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    # 求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test', sess.graph)
    for epoch in range(5):
        
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.0})
        
        train_writer.add_summary(summary, epoch)
        train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.0})
        summary, acc = sess.run([merged, accuracy], feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        test_writer.add_summary(summary, epoch)
        
        print "iter: " + str(epoch) + ", train accuracy: " + str(train_acc) + ", iter: " + str(epoch) + ", test accuracy: " + str(acc)
        
        

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

iter: 0, train accuracy: 0.99, iter: 0, test accuracy: 0.9847
iter: 1, train accuracy: 1.0, iter: 1, test accu